# 實戰：網路書店新書排行榜

## 取得新書分類排行榜資料

In [ ]:
def showkind(url,kind):
  html = requests.get(url).text
  soup = BeautifulSoup(html,'lxml') 
  try:
    # 該分類共有多少頁
    pages=int(soup.select('.cnt_page span')[0].text)  
    print("共有",pages,"頁")
    for page in range(1,pages+1):
      pageurl=url + '&page=' + str(page).strip()
      print("第",page,"頁",pageurl)
      showpage(pageurl,kind)
  except:  # 沒有分頁的處理
    showpage(url,kind)   

def showpage(url,kind):
  html = requests.get(url).text
  soup = BeautifulSoup(html,'lxml') 
  #近期新書、在 class="mod type02_m012 clearfix" 中
  res = soup.find('div',{'class':'mod type02_m012 clearfix'})
  items=res.select('.item')  # 所有 item
  n=0  # 計算該分頁共有多少本書
  for item in items:
    msg=item.select('.msg')[0] 
    src=item.select('a img')[0]["src"]
    title=msg.select('a')[0].text  #書名
    imgurl=src.split("?i=")[-1].split("&")[0] #圖片網址
    author=msg.select('a')[1].text #作者
    publish=msg.select('a')[2].text #出版社
    date=msg.find('span').text.split("：")[-1] #出版日期
    onsale=item.select('.price .set2')[0].text #優惠價
    content=item.select('.txt_cont')[0].text\
      .replace(" ","").strip()  #內容
    print("\n分類：" + kind)  
    print("書名：" + title)   
    print("圖片網址：" + imgurl)  
    print("作者：" + author)      
    print("出版社：" + publish)  
    print("出版日期：" + date) 
    print(onsale) # 優惠價 
    print("內容：" + content)     
    n+=1
    print("n=",n)

def twobyte(kindno):
  if kindno<10:
    kindnostr="0"+str(kindno)
  else:
    kindnostr=str(kindno) 
  return kindnostr

#主程式
import requests
from bs4 import BeautifulSoup 
kindno=1  # 要下載的分類，預設為第1分類：文學小說
homeurl = 'http://www.books.com.tw/web/books_nbtopm_01/?o=5&v=1'
mode="/?o=5&v=1" #顯示模式：直式  排序依：暢銷度
url="http://www.books.com.tw/web/books_nbtopm_" 
html = requests.get(homeurl).text
soup = BeautifulSoup(html,'lxml') 

#中文書新書分類，取得分類資訊
res = soup.find('div', class_='mod_b type02_l001-1 clearfix')
hrefs=res.select("a")

kindno=int(input("請輸入要下載的分類："))
if 0 < kindno <= len(hrefs):
  kind = hrefs[kindno-1].text #分類名稱
  print("下載的分類編號：{}   分類名稱：{}" .format(kindno,kind))
  # 下載指定的分類
  kindurl = url + twobyte(kindno) + mode # 分類網址  
  print(kindurl)
  showkind(kindurl, kind) # 顯示該分類所有書籍
else:
  print("分類不存在!")     

## 將資料儲存到 Google 試算表

### 安裝相關模組


In [ ]:
!pip install gspread oauth2client

### 專題完整的程式碼

In [ ]:
def showkind(url,kind):
  html = requests.get(url).text
  soup = BeautifulSoup(html,'html.parser') 
  try:
    # 該分類共有多少頁
    pages=int(soup.select('.cnt_page span')[0].text)
    print("共有",pages,"頁")
    for page in range(1,pages+1):
      pageurl=url + '&page=' + str(page).strip()
      print("第",page,"頁",pageurl)
      showpage(pageurl,kind)
  except:  # 沒有分頁的處理
    showpage(url,kind)        

def showpage(url,kind):
  html = requests.get(url).text 
  soup = BeautifulSoup(html,'html.parser') 
  #近期新書、在 class="mod type02_m012 clearfix" 中
  res = soup.find('div',{'class':'mod type02_m012 clearfix'})
  items=res.select('.item')  # 所有 item
  n=0  # 計算該分頁共有多少本書
  for item in items:
    msg=item.select('.msg')[0] 
    src=item.select('a img')[0]["src"]
    title=msg.select('a')[0].text  #書名
    imgurl=src.split("?i=")[-1].split("&")[0] #圖片網址
    author=msg.select('a')[1].text #作者
    publish=msg.select('a')[2].text #出版社
    date=msg.find('span').text.split("：")[-1] #出版日期
    onsale=item.select('.price .set2')[0].text #優惠價
    content=item.select('.txt_cont')[0].text.replace(" ","").strip() #內容
    # 將資料加入 list1 串列中
    listdata=[kind,title,imgurl,author,publish,date,onsale,content]
    list1.append(listdata)
    n+=1
    print("n=",n)

def twobyte(kindno):
  if kindno<10:
    kindnostr="0"+str(kindno)
  else:
    kindnostr=str(kindno) 
  return kindnostr

def auth_gss_client(path, scopes): #建立憑證
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
                 path,scopes)
  return gspread.authorize(credentials)  

#主程式
import requests
from bs4 import BeautifulSoup
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from time import sleep

auth_json_path = 'PythonUpload.json' # 自己的 json 金鑰檔
gss_scopes = ['https://spreadsheets.google.com/feeds']
gss_client = auth_gss_client(auth_json_path, gss_scopes) #連線

spreadsheet_key = '1edTW5DMS9ZC01DL7dU3fIhEEMuJolvx_X9c6Dhm2IHA' 
sheet = gss_client.open_by_key(spreadsheet_key).sheet1 #開啟工作表
sheet.clear() # 清除工作表內容

list1=[]    
kindno=1  # 要下載的分類，預設為第 1分類：文學小說
homeurl = 'http://www.books.com.tw/web/books_nbtopm_01/?o=5&v=1'
mode="/?o=5&v=1" #顯示模式：直式  排序依：暢銷度
url="https://www.books.com.tw/web/books_nbtopm_" 
html = requests.get(homeurl).text
soup = BeautifulSoup(html,'html.parser') 
#中文書新書分類，取得分類資訊
res = soup.find('div',{'class':'mod_b type02_l001-1 clearfix'})
hrefs=res.select("a")
kindno=int(input("請輸入要下載的分類："))
if 0 < kindno <= len(hrefs):
  kind=hrefs[kindno-1].text #分類名稱
  print("下載的分類編號：{}   分類名稱：{}" .format(kindno,kind))
  # 下載指定的分類
  kindurl=url + twobyte(kindno) + mode # 分類網址  
  showkind(kindurl,kind) # 顯示該分類所有書籍    
  # 儲存 Google 試算表
  print("資料寫入雲端 Google 試算表中，請等侯幾分鐘!")
  listtitle=["分類","書名","圖片網址","作者","出版社",
             "出版日期","優惠價","內容"]
  sheet.append_row(listtitle)  # 標題
  for item1 in list1: #資料
    sheet.append_row(item1)
    sleep(1.2) # 必須加上適當的 delay      
else:
  print("分類不存在!") 
print("資料儲存完畢!")    

請輸入要下載的分類：2
下載的分類編號：2   分類名稱：商業理財
共有 3 頁
第 1 頁 https://www.books.com.tw/web/books_nbtopm_02/?o=5&v=1&page=1
n= 1
n= 2
n= 3
n= 4
n= 5
n= 6
n= 7
n= 8
n= 9
n= 10
n= 11
n= 12
n= 13
n= 14
n= 15
n= 16
n= 17
n= 18
n= 19
n= 20
n= 21
n= 22
n= 23
n= 24
n= 25
n= 26
n= 27
n= 28
n= 29
n= 30
n= 31
n= 32
n= 33
n= 34
n= 35
n= 36
n= 37
n= 38
n= 39
n= 40
n= 41
n= 42
n= 43
n= 44
n= 45
n= 46
n= 47
n= 48
n= 49
n= 50
n= 51
n= 52
n= 53
n= 54
n= 55
n= 56
n= 57
n= 58
n= 59
n= 60
n= 61
n= 62
n= 63
n= 64
n= 65
n= 66
n= 67
n= 68
n= 69
n= 70
n= 71
n= 72
n= 73
n= 74
n= 75
n= 76
n= 77
n= 78
n= 79
n= 80
n= 81
n= 82
n= 83
n= 84
n= 85
n= 86
n= 87
n= 88
n= 89
n= 90
n= 91
n= 92
n= 93
n= 94
n= 95
n= 96
n= 97
n= 98
n= 99
n= 100
第 2 頁 https://www.books.com.tw/web/books_nbtopm_02/?o=5&v=1&page=2
n= 1
n= 2
n= 3
n= 4
n= 5
n= 6
n= 7
n= 8
n= 9
n= 10
n= 11
n= 12
n= 13
n= 14
n= 15
n= 16
n= 17
n= 18
n= 19
n= 20
n= 21
n= 22
n= 23
n= 24
n= 25
n= 26
n= 27
n= 28
n= 29
n= 30
n= 31
n= 32
n= 33
n= 34
n= 35
n= 36
n= 37
n= 38
n= 39
n= 40
n=